## Style block (delete header)

[Einhverjar góðar upplýsingar um page-styling](https://stackoverflow.com/questions/3341485/how-to-make-a-html-page-in-a4-paper-size-pages)

<style type="text/css">
* {
    max-width: 800px;
}
</style>

<hr>

# Verkefni II

### Motion Control in Computer-Aided Modeling

##### Reality Check 5, Numerical Analysis 2nd ed., Sauer.
<hr>

#### Erling Óskar Kristjánsson, eok4@hi.is

#### Davíð Freyr Björnsson, dfb2@hi.is
<hr>

##### Námskeið: Töluleg Greining (Stæ405G)

##### Stofnun: Háskóli Íslands

##### Umsjónamaður: Sigurður Freyr Hafstein

##### Vor 2019
<hr>

## Introduction 

Computer-aided modeling and manufacturing requires precise control of spatial position
along a prescribed motion path. We will illustrate the use of Adaptive Quadrature to solve
a fundamental piece of the problem: equipartition, or the division of an arbitrary path into
equal-length subpaths.

In numerical machining problems, it is preferable to maintain constant speed along the
path. During each second, progress should be made along an equal length of the machine–
material interface. In other motion planning applications, including computer animation,
more complicated progress curves may be required: A hand reaching for a doorknob might 
begin and end with low velocity and have higher velocity in between. Robotics and virtual
reality applications require the construction of parametrized curves and surfaces to be
navigated. Building a table of small equal increments in path distance is often a necessary
first step.

Assume that a parametric path P = $\{x(t),\ y(t) \ | \  0 \leq t \leq 1\}$ is given. Figure 5.6 shows
the example path

$$
P =
\begin{cases}
x(t) = 0.5 + 0.3t + 3.9t^2 − 4.7t^3, \\
y(t) = 1.5 + 0.3t + 0.9t^2 − 2.7t^3
\end{cases}
$$

Bézier curve defined by the four points $(0.5,1.5)$, $(0.6,1.6)$, $(2,2)$, $(0,0)$. 
Points defined by evenly spaced parameter values $t = 0, 1/4, 1/2, 3/4, 1$ are shown. 

![(May need to host pictures like this online and supply a https link) Figure 5.6](img/Figure5_6.JPG)

Note that even spacing in parameter does not imply even spacing in arc length. 
Your goal is to apply quadrature methods to divide this path into $n$ equal lengths.

Recall from calculus that the arc length of the path from $t_1$ to $t_2$ is

$$
\int_{t_1}^{t_2} \sqrt{x'(t)^2+y'(t)^2}dt
$$

Only rarely does the integral yield a closed-form expression, and normally an Adaptive
Quadrature technique is used to control the parametrization of the path.

<hr>

## Suggested Activity 1

Write a Python function that uses Adaptive Quadrature to compute the arc length from $t_1 = 0$ to $t_2 = T$ for a given $T \leq 1$.

** Ready as far as $x'(t)$ and $y'(t)$ are supplied. The given $P(x(t),y(t))$ was differentiated manually. **

In [9]:
import numpy as np

# tol gives correct to 8 decimal places
def ComputeArcLength(f, T, t1=0, tol = 0.5e-8):
    return AdaptiveIntegration(f, t1, T, tol)

# Just a function that calls another function
def AdaptiveIntegration(f, a, b, tol):
    return AdaptiveQuadratureS(f, f(a), f(b), a, b, tol)

# Computes the AdaptiveQuadrature
def AdaptiveQuadratureS(f, fa, fb, a, b, tol):
    c = (a+b)/2
    h = b-a
    fc = f(c)
    S_ab = (h/2)*(fa+fb)
    S_ac = (h/4)*(fa+fc)
    S_cb = (h/4)*(fc+fb)
    if np.abs(S_ab - S_cb - S_ac) < 3*tol and h < 1e-2:
        return (S_ac + S_cb)
    else:
        return ( AdaptiveQuadratureS(f, fa, fc, a, c, tol/2) +
                AdaptiveQuadratureS(f, fc, fb, c, b, tol/2) )
            

def SqrtOfFunctionsSquared(x,y):
    return lambda t: np.sqrt(x(t)**2 + y(t)**2)
    

# Possible useful
def ThreePointCenteredDifferenceFormula(f, a=0, h=1e-10):
    return (f(a+h) - f(a-h))/(2*h)

# example functions and usage of computeArcLength
dxdt = lambda t: 0.3 + 7.8*t - 14.1*t**2
dydt = lambda t: 0.3 + 1.8*t - 8.1*t**2
fsa1 = SqrtOfFunctionsSquared(dxdt, dydt)

SA1 = ComputeArcLength(fsa1, 1)
print('The arc length is: ' + str(SA1))

The arc length is: 2.49524674832873


<hr>

## Suggested Activity 2

Write a program that, for any input $s$ between $0$ and $1$, finds the parameter $t^∗(s)$
that is $s$ of the way along the curve. In other words, the arc length from $t = 0$
to $t = t^∗(s)$ divided by the arc length from $t = 0$ to $t = 1$ should be equal to $s$.

**In other other words: **

$$ s = \frac{\int_{0}^{t^*(s)} \sqrt{x'(t)^2+y'(t)^2}dt}{ \int_{0}^{1} \sqrt{x'(t)^2+y'(t)^2}dt}$$

Use the Bisection Method to locate the point $t^∗(s)$ to three correct decimal places. 

1. What function is being set to zero?
$$ s \int_{0}^{1} \sqrt{x'(t)^2+y'(t)^2}dt - \int_{0}^{t^*(s)} \sqrt{x'(t)^2+y'(t)^2}dt = 0$$

3. What bracketing interval should be used to start the Bisection Method?

Good question. I'm just using [0, 1], since for $t^*(s) = 0$ it follows that $s=0$ and when $t^*(s) = 1$ it follows that $s=1$. So this range of $t^*(s)$ covers all the possible values of s.

In [10]:
# Notkun:   BisectionMethod(f,a,b)
# Fyrir:    f is a continuous fucntion on the interval [a,b]
#           a < b are numbers such that f(a)*f(b) < 0
#           tol is a number
# Eftir:    c is the root of f, i.e f(c) = 0
def BisectionMethod(f,a,b,tol=0.5e-4):
    fa=f(a)
    fb=f(b)
    while (b-a)/2 > tol:
        c=(a+b)/2
        fc=f(c)
        if fa*fc <= 0:
            b=c
            fb=fc
        else:
            a=c
            fa=fc
    return c

def tStarOfS(f, s):
    arcLen0to1 = ComputeArcLength(f, 1)
    g = lambda b: (s * arcLen0to1 - ComputeArcLength(f, b))
    return BisectionMethod(g, 0, 1)


s = 0.5
ts = tStarOfS(fsa1, s)
print('The optimal value of t for ' + str(s) + ' is ' + str(ts))

# We can verify that with:
print('Verified:', (np.abs(ComputeArcLength(fsa1, ts) / ComputeArcLength(fsa1, 1) - s) < 0.001) )

# Lower tolerances to reduce computation time




The optimal value of t for 0.5 is 0.80059814453125
Verified: True


<hr>

## Suggested Activity 3

Equipartition (_split into parts of equal length_) the path of Figure 5.6 into $n$ subpaths of equal length, for $n = 4$ and $n = 20$. Plot analogues of Figure 5.6, showing the equipartitions. If your computations are too slow, consider speeding up the Adaptive Quadrature with Simpson’s Rule, as suggested in Computer Problem 5.4.2.

We use the methodology from the previous example. First we implement an efficient version of the Adaptive Quadrature with Simpson's Rule, shows below.

Let us consider the scenario where $n = 4$: 
- We know that the length of the curve is approximately $2.49524674$, so in this scenario we want each part to be of length $0.623811685$.



<hr>

## Suggested Activity 4

Replace the Bisection Method in Step 2 with Newton’s Method, and repeat Steps 2 and 3.

1. What is the derivative needed?
2. What is a good choice for the initial guess?
3. Is computation time decreased by this replacement?